In [1]:
import gzip
import io
from collections import Counter
from threading import Thread
import numpy as np
import pandas as pd
import requests
from pandas import DataFrame

In [2]:
url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2018/full.csv.gz"
req = requests.get(url)

In [3]:
t = req.content

In [4]:
s = gzip.decompress(t)

In [5]:
df = pd.read_csv(io.BytesIO(s))

C:\Users\dargo\AppData\Local\Temp\ipykernel_19344\798997524.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.BytesIO(s))


In [6]:
# suppression valeur fonciere nulle
df.dropna(subset=['valeur_fonciere'], inplace=True)
df.dropna(subset=['longitude', 'latitude'], inplace=True)

In [7]:
# Selection des colonnes utiles
dataset = df[['id_mutation',
              'date_mutation',
              'nature_mutation',
              'adresse_nom_voie',
              'code_postal',
              'code_commune',
              'code_departement',
              'nombre_lots',
              'type_local',
              'surface_reelle_bati',
              'nombre_pieces_principales',
              'code_nature_culture',
              'code_nature_culture_speciale',
              'surface_terrain',
              'valeur_fonciere',
              'longitude',
              'latitude'
              ]]


In [8]:
del df

# Cleaning

In [9]:
dataset.shape

(3201638, 17)

### Date mutation cleaning

In [10]:
# Passage de la colonnes date mutation en datetime puis en seconde pour avoir des int
dataset.loc[:, 'date_mutation'] = pd.to_datetime(dataset['date_mutation'], format='mixed')
dataset.loc[:, 'date_mutation'] = dataset['date_mutation'].apply(lambda x: x.timestamp())

In [11]:
# Remplissage des valeurs NaN nature_mutation et labelisation
dataset.loc[:, 'nature_mutation'] = dataset.nature_mutation.fillna("")

NM = preprocessing.LabelEncoder()
NM.fit(dataset['nature_mutation'])
dataset.loc[:, 'nature_mutation'] = NM.transform(dataset['nature_mutation'])

### Adresse nom voie cleaning

In [12]:
abrv_voie = pd.read_csv(filepath_or_buffer='Data_Files/ABREVIATION_VOIE.csv', sep=',').values

In [13]:
dataset.loc[:, 'adresse_nom_voie'] = dataset.adresse_nom_voie.fillna("")

codex_voie = list()
for i in dataset.adresse_nom_voie.values:

    list_nom_rue = i.split(' ')
    premier_valeur_liste = list_nom_rue[0]

    if premier_valeur_liste in abrv_voie:
        codex_voie.append(premier_valeur_liste)
    else:
        codex_voie.append('AUTRE')

dataset.loc[:, 'prefixe_voie'] = codex_voie

CV = preprocessing.LabelEncoder()
CV.fit(dataset.prefixe_voie)
dataset.loc[:,'prefixe_voie'] = CV.transform(dataset.prefixe_voie)

### Multiple easy Cleaning

In [14]:
dataset.loc[:, 'code_postal'] = dataset.code_postal.fillna(0)

In [15]:
dataset.loc[:, 'surface_reelle_bati'] = dataset.surface_reelle_bati.fillna(0)
dataset.loc[:, 'surface_terrain'] = dataset.surface_terrain.fillna(0)
dataset.loc[:, 'type_local'] = dataset.type_local.fillna('Autre')
dataset.loc[:, 'nombre_pieces_principales'] = dataset.nombre_pieces_principales.fillna(0)

In [16]:
dataset.loc[:, 'code_nature_culture'] = dataset.code_nature_culture.fillna("")
dataset.loc[:, 'code_nature_culture_speciale'] = dataset.code_nature_culture_speciale.fillna("")

### Latitude & longitude

# Construction data set modele

In [17]:
classe_liste_nature_mutation = list(dict.fromkeys(list(dataset.nature_mutation.values)))

In [18]:
classe_liste_code_type_local = list(dict.fromkeys(list(dataset.type_local.values)))

In [19]:
classe_liste_prefixe_voie = list(dict.fromkeys(codex_voie))

In [20]:
classe_liste_code_culture = pd.read_csv(filepath_or_buffer='Data_Files/CODE_CULTURE.csv', sep=',')
classe_liste_code_culture = list(classe_liste_code_culture['Code_nature_culture'].to_dict().values())

In [21]:
classe_liste_code_culture_spe = pd.read_csv(filepath_or_buffer='Data_Files/CODE_CULTURE_SPECIALE.csv', sep=',')
classe_liste_code_culture_spe = list(classe_liste_code_culture_spe['CODE_CULTURE_SPECIALE'].to_dict().values())

In [22]:
Nom_colonnes_from_dataset = [
    'date_mutation',
    'code_postal',
    'code_commune',
    'code_departement',
    'nombre_lots',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'surface_terrain',
    'valeur_fonciere',
    'longitude',
    'latitude'
]

In [23]:
colonnes = Nom_colonnes_from_dataset + classe_liste_code_type_local + classe_liste_prefixe_voie + classe_liste_nature_mutation + classe_liste_code_culture + classe_liste_code_culture_spe
colonnes = np.array(colonnes)
del  classe_liste_nature_mutation, classe_liste_code_type_local, classe_liste_prefixe_voie, classe_liste_code_culture, classe_liste_code_culture_spe

assert 1 == len(pd.unique(TEST.date_mutation))
assert 1 == len(pd.unique(TEST.valeur_fonciere))
assert 1 == len(pd.unique(TEST.code_postal))
assert 1 == len(pd.unique(TEST.code_commune))
assert 1 == len(pd.unique(TEST.code_departement))
assert len(ajout_culture_spe) == len(classe_liste_code_culture_spe)
assert len(ajout_culture) == len(classe_liste_code_culture)
assert len(ajout_nature_mutation) == len(classe_liste_nature_mutation)
assert len(ajout_abbrev_voie) == len(classe_liste_prefixe_voie)
assert len(ajout_code_type) == len(classe_liste_code_type_local)


# Dataset creation

In [24]:
test = dataset.loc[0:100000]

In [25]:
list_id_mutation = list(dict.fromkeys(test.id_mutation.values))

In [26]:
class My_thread(Thread):
    def __init__(self, id, dataset, colonnes):
        super(My_thread,self).__init__()
        self.id = id
        self.dataset = dataset
        self.colonnes = colonnes
        self.retour = pd.DataFrame()
    
    def run(self) -> None:
        tmp_dataset = self.dataset
        index = tmp_dataset.index[0]
        tmp_dict = dict.fromkeys(self.colonnes)
    
        tmp_dict['date_mutation'] = [tmp_dataset.loc[index, 'date_mutation']]
        tmp_dict['code_postal'] = [tmp_dataset.loc[index, 'code_postal']]
        tmp_dict['code_commune'] = [tmp_dataset.loc[index, 'code_commune']]
        tmp_dict['code_departement'] = [tmp_dataset.loc[index, 'code_departement']]
        tmp_dict['nombre_lots'] = [tmp_dataset.loc[index, 'nombre_lots'].sum()]
        tmp_dict['surface_reelle_bati'] = [tmp_dataset.loc[index, 'surface_reelle_bati'].sum()]
        tmp_dict['nombre_pieces_principales'] = [tmp_dataset.loc[index, 'nombre_pieces_principales'].sum()]
        tmp_dict['surface_terrain'] = [tmp_dataset.loc[index, 'surface_terrain'].sum()]
        tmp_dict['valeur_fonciere'] = [tmp_dataset.loc[index, 'valeur_fonciere']]
        tmp_dict['longitude'] = [tmp_dataset.loc[index, 'longitude']]
        tmp_dict['latitude'] = [tmp_dataset.loc[index, 'latitude']]
    
        # valeur classe code type local
        count_type_local = Counter(tmp_dataset.loc[:, 'type_local'])
        for type_local in count_type_local:
            tmp_dict[type_local] = count_type_local[type_local]
    
        # valeur prefixe voie
        count_prefixe_voie = Counter(tmp_dataset.loc[:, 'prefixe_voie'])
        for abbrev_voie in count_prefixe_voie:
            tmp_dict[abbrev_voie] = count_prefixe_voie[abbrev_voie]
    
        # valeur classe nature mutation
        count_nature_mutation = Counter(tmp_dataset.loc[:, 'nature_mutation'])
        for nat_mutation in count_nature_mutation:
            tmp_dict[nat_mutation] = count_nature_mutation[nat_mutation]
    
        # valeur classe culture
        count_type_culture = Counter(tmp_dataset.loc[:, 'code_nature_culture'])
        for culture in count_type_culture:
            tmp_dict[culture] = count_type_culture[culture]
    
        # valeur classe culture
        count_type_culture_spe = Counter(tmp_dataset.loc[:, 'code_nature_culture_speciale'])
        for culture_spe in count_type_culture_spe:
            tmp_dict[culture_spe] = count_type_culture_spe[culture_spe]
        
        self.retour = pd.DataFrame.from_dict(tmp_dict)

In [27]:
th_liste = list(range(len(list_id_mutation)))
Data = pd.DataFrame()

In [28]:
for i, id in enumerate(list_id_mutation):
    tmp = test.loc[test.id_mutation == id,:]
    th_liste[i] : My_thread = My_thread(id, tmp, colonnes)
    th_liste[i].start()
    test = test.drop(test[test.id_mutation == id].index)

In [29]:
while len(th_liste) != 0:
    th_liste[0].join()
    Data = pd.concat([Data, th_liste[0].retour], axis=0)
    del th_liste[0]

KeyboardInterrupt: 

In [ ]:
Data = Data.fillna(0)

# autre methode

In [ ]:
Data = pd.DataFrame()
for i, id in enumerate(list_id_mutation):

    tmp_dataset = test.loc[dataset.id_mutation == id]
    test = test.drop(list(test.loc[test.id_mutation == id, :].index))
    index = tmp_dataset.index[0]

    tmp_dict = dict.fromkeys(colonnes)

    tmp_dict['date_mutation'] = [tmp_dataset.loc[index, 'date_mutation']]
    tmp_dict['code_postal'] = [tmp_dataset.loc[index, 'code_postal']]
    tmp_dict['code_commune'] = [tmp_dataset.loc[index, 'code_commune']]
    tmp_dict['code_departement'] = [tmp_dataset.loc[index, 'code_departement']]
    tmp_dict['nombre_lots'] = [tmp_dataset.loc[index, 'nombre_lots'].sum()]
    tmp_dict['surface_reelle_bati'] = [tmp_dataset.loc[index, 'surface_reelle_bati'].sum()]
    tmp_dict['nombre_pieces_principales'] = [tmp_dataset.loc[index, 'nombre_pieces_principales'].sum()]
    tmp_dict['surface_terrain'] = [tmp_dataset.loc[index, 'surface_terrain'].sum()]
    tmp_dict['valeur_fonciere'] = [tmp_dataset.loc[index, 'valeur_fonciere']]
    tmp_dict['longitude'] = [tmp_dataset.loc[index, 'longitude']]
    tmp_dict['latitude'] = [tmp_dataset.loc[index, 'latitude']]

    # valeur classe code type local
    tmp_code_type_local = tmp_dataset.loc[:, 'type_local']
    for code_type in classe_liste_code_type_local:
        tmp_dict[code_type] = [tmp_code_type_local.loc[tmp_code_type_local == code_type].size]

    # valeur prefixe voie
    tmp_abbrev_voie = tmp_dataset.loc[:, 'prefixe_voie']
    for abbrev_voie in classe_liste_prefixe_voie:
        tmp_dict[abbrev_voie] = [tmp_abbrev_voie.loc[tmp_abbrev_voie == abbrev_voie].size]

    # valeur classe nature mutation
    tmp_nature_mutation = tmp_dataset.loc[:, 'nature_mutation']
    for nat_mutation in classe_liste_nature_mutation:
        tmp_dict[nat_mutation] = [tmp_nature_mutation.loc[tmp_nature_mutation == nat_mutation].size]

    # valeur classe culture
    tmp_culture = tmp_dataset.loc[:, 'code_nature_culture']
    for culture in classe_liste_code_culture:
        tmp_dict[culture] = [tmp_culture.loc[tmp_culture == culture].size]

    # valeur classe culture spe
    tmp_culture_spe = tmp_dataset.loc[:, 'code_nature_culture_speciale']

    for culture_spe in classe_liste_code_culture_spe:
        tmp_dict[culture_spe] = [tmp_culture_spe.loc[tmp_culture_spe == culture_spe].size]

    Data = pd.concat([Data, pd.DataFrame.from_dict(tmp_dict)], ignore_index=True, axis=0)

In [ ]:
Data = pd.DataFrame()
for i, id in enumerate(list_id_mutation):

    tmp_dataset = test.loc[dataset.id_mutation == id]
    index = tmp_dataset.index[0]

    tmp_dict = dict.fromkeys(colonnes)

    tmp_dict['date_mutation'] = [tmp_dataset.loc[index, 'date_mutation']]
    tmp_dict['code_postal'] = [tmp_dataset.loc[index, 'code_postal']]
    tmp_dict['code_commune'] = [tmp_dataset.loc[index, 'code_commune']]
    tmp_dict['code_departement'] = [tmp_dataset.loc[index, 'code_departement']]
    tmp_dict['nombre_lots'] = [tmp_dataset.loc[index, 'nombre_lots'].sum()]
    tmp_dict['surface_reelle_bati'] = [tmp_dataset.loc[index, 'surface_reelle_bati'].sum()]
    tmp_dict['nombre_pieces_principales'] = [tmp_dataset.loc[index, 'nombre_pieces_principales'].sum()]
    tmp_dict['surface_terrain'] = [tmp_dataset.loc[index, 'surface_terrain'].sum()]
    tmp_dict['valeur_fonciere'] = [tmp_dataset.loc[index, 'valeur_fonciere']]
    tmp_dict['longitude'] = [tmp_dataset.loc[index, 'longitude']]
    tmp_dict['latitude'] = [tmp_dataset.loc[index, 'latitude']]

    # valeur classe code type local
    count_type_local = Counter(tmp_dataset.loc[:, 'type_local'])
    for type_local in count_type_local:
        tmp_dict[type_local] = count_type_local[type_local]

    # valeur prefixe voie
    count_prefixe_voie = Counter(tmp_dataset.loc[:, 'prefixe_voie'])
    for abbrev_voie in count_prefixe_voie:
        tmp_dict[abbrev_voie] = count_prefixe_voie[abbrev_voie]

    # valeur classe nature mutation
    count_nature_mutation = Counter(tmp_dataset.loc[:, 'nature_mutation'])
    for nat_mutation in count_nature_mutation:
        tmp_dict[nat_mutation] = count_nature_mutation[nat_mutation]

    # valeur classe culture
    count_type_culture = Counter(tmp_dataset.loc[:, 'code_nature_culture'])
    for culture in count_type_culture:
        tmp_dict[culture] = count_type_culture[culture]

    # valeur classe culture
    count_type_culture_spe = Counter(tmp_dataset.loc[:, 'code_nature_culture_speciale'])
    for culture_spe in count_type_culture_spe:
        tmp_dict[culture_spe] = count_type_culture_spe[culture_spe]

    Data = pd.concat([Data, pd.DataFrame.from_dict(tmp_dict)], ignore_index=True, axis=0)

In [ ]:
def traitement_id(id: str, dataset: DataFrame, colonnes: np.array, ) -> DataFrame:
    tmp_dataset = dataset.loc[dataset.id_mutation == id]
    index = tmp_dataset.index[0]

    tmp_dict = dict.fromkeys(colonnes)

    tmp_dict['date_mutation'] = [tmp_dataset.loc[index, 'date_mutation']]
    tmp_dict['code_postal'] = [tmp_dataset.loc[index, 'code_postal']]
    tmp_dict['code_commune'] = [tmp_dataset.loc[index, 'code_commune']]
    tmp_dict['code_departement'] = [tmp_dataset.loc[index, 'code_departement']]
    tmp_dict['nombre_lots'] = [tmp_dataset.loc[index, 'nombre_lots'].sum()]
    tmp_dict['surface_reelle_bati'] = [tmp_dataset.loc[index, 'surface_reelle_bati'].sum()]
    tmp_dict['nombre_pieces_principales'] = [tmp_dataset.loc[index, 'nombre_pieces_principales'].sum()]
    tmp_dict['surface_terrain'] = [tmp_dataset.loc[index, 'surface_terrain'].sum()]
    tmp_dict['valeur_fonciere'] = [tmp_dataset.loc[index, 'valeur_fonciere']]
    tmp_dict['longitude'] = [tmp_dataset.loc[index, 'longitude']]
    tmp_dict['latitude'] = [tmp_dataset.loc[index, 'latitude']]

    # valeur classe code type local
    count_type_local = Counter(tmp_dataset.loc[:, 'type_local'])
    for type_local in count_type_local:
        tmp_dict[type_local] = count_type_local[type_local]

    # valeur prefixe voie
    count_prefixe_voie = Counter(tmp_dataset.loc[:, 'prefixe_voie'])
    for abbrev_voie in count_prefixe_voie:
        tmp_dict[abbrev_voie] = count_prefixe_voie[abbrev_voie]

    # valeur classe nature mutation
    count_nature_mutation = Counter(tmp_dataset.loc[:, 'nature_mutation'])
    for nat_mutation in count_nature_mutation:
        tmp_dict[nat_mutation] = count_nature_mutation[nat_mutation]

    # valeur classe culture
    count_type_culture = Counter(tmp_dataset.loc[:, 'code_nature_culture'])
    for culture in count_type_culture:
        tmp_dict[culture] = count_type_culture[culture]

    # valeur classe culture
    count_type_culture_spe = Counter(tmp_dataset.loc[:, 'code_nature_culture_speciale'])
    for culture_spe in count_type_culture_spe:
        tmp_dict[culture_spe] = count_type_culture_spe[culture_spe]

    return pd.DataFrame.from_dict(tmp_dict).fillna(0)

In [ ]:
# Sortie de la colonne resultat

## Y = df.valeur_fonciere